In [1]:
import os

# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran4

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [9]:
# What occupation had the highest salary?
df.orderBy(df["salary"].desc()).select("occupation", "salary").limit(1).show()

+-----------------+------+
|       occupation|salary|
+-----------------+------+
|Medical Physicist|    90|
+-----------------+------+



In [10]:
# What occupation had the lowest salary?
df.orderBy(df["salary"].asc()).select("occupation", "salary").limit(1).show()

+--------------+------+
|    occupation|salary|
+--------------+------+
|Window Dresser|    65|
+--------------+------+



In [11]:
# What is the mean salary of this dataset?
from pyspark.sql.functions import mean
df.select(mean("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     77.738|
+-----------+



In [12]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import max, min
df.select(max("salary"), min("salary")).show()

+-----------+-----------+
|max(salary)|min(salary)|
+-----------+-----------+
|         90|         65|
+-----------+-----------+



In [16]:
# Show all of the occupations where salaries were above 80k
df.filter("salary>80").select(["occupation"]).show()

+--------------------+
|          occupation|
+--------------------+
|              Hawker|
|       Choreographer|
|          Millwright|
|   Medical Physicist|
|           Scientist|
|     Claims Adjustor|
| Planning Technician|
|       Booking Clerk|
|      Sub-Postmaster|
|        Shelf Filler|
|             Chemist|
|        Betting Shop|
|     Hire Car Driver|
|    Heating Engineer|
|    Vehicle Assessor|
|   Building Surveyor|
|Advertising Contr...|
|   Medical Physicist|
|            Labourer|
|   Technical Analyst|
+--------------------+
only showing top 20 rows



In [ ]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this

In [19]:
df.groupBy("academic_degree").avg().select("academic_degree", "avg(age)", "avg(height_meter)").show()

+---------------+------------------+------------------+
|academic_degree|          avg(age)| avg(height_meter)|
+---------------+------------------+------------------+
|            PhD| 43.15976331360947|1.7438165680473379|
|         Master|43.139318885448915|1.7549226006191951|
|       Bachelor| 42.51032448377581| 1.757227138643069|
+---------------+------------------+------------------+

